# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from os import listdir, walk
from os.path import isfile, join

# Load Predicted Data

In [66]:
%run activity_summary.ipynb
%run ../ddc/preprocessing.ipynb
%run ../ddc/load_dataset.ipynb

In [3]:
subj_range = np.hstack((np.arange(2001,2002),np.arange(3001,3006)))
all_patients = [str(i) for i in subj_range]

In [4]:
datapath = '../../DDC_Data/'
basepath = '../../'

In [5]:
csv_path = datapath + 'prediction/predicted_data_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
df_all_p_sorted = pd.read_csv(csv_path)

In [6]:
df_date = df_all_p_sorted.copy()
df_date['date'] = df_date['timestamp'].apply(lambda x: x.split(' ')[0])
df_date['time'] = df_date['timestamp'].apply(lambda x: x.split(' ')[1])

cols = ['ID','date','time','x','y','z','HR','y_pred']
df_date = df_date[cols]

# Summarize Data

In [68]:
df_summary_all, df_act_period = get_df_summary_all(all_patients, df_date)

/Users/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [69]:
cols = ['ID', 'date', 'from', 'to', 'y_pred']
df_act_period = df_act_period[cols]
df_act_period = df_act_period.reset_index(drop=True)

df_summary_all = df_summary_all.reset_index(drop=True)

In [70]:
df_summary_all

ID        date    from      to     sit   sleep   stand    walk   total  \
0  2001  2019-01-16  13:0:0  14:0:0  0:0:35  0:0:00  0:0:36  0:0:00  0:1:11   
1  2001  2019-01-16  14:0:0  15:0:0  0:1:07  0:1:10  0:4:03  0:3:04  0:9:24   
2  3001  2019-03-05  10:0:0  11:0:0  0:1:55  0:1:09  0:3:02  0:1:04  0:7:10   
3  3002  2019-03-05  12:0:0  13:0:0  0:1:06  0:1:10  0:2:48  0:0:30  0:5:34   
4  3003  2019-03-05  12:0:0  13:0:0  0:1:28  0:1:15  0:2:09  0:1:27  0:6:19   
5  3004  2019-03-05  12:0:0  13:0:0  0:1:53  0:1:18  0:2:44  0:0:56  0:6:51   
6  3005  2019-03-05  12:0:0  13:0:0  0:0:01  0:0:00  0:0:30  0:0:00  0:0:31   
7  3005  2019-03-05  13:0:0  14:0:0  0:1:44  0:1:17  0:1:47  0:0:59  0:5:47   

   sit count  sleep count  stand count  walk count  inactive count  \
0          1            0            2           0               1   
1         16            6           88          91              22   
2         12            9           32          19              21   
3          6            4           25          20              10   
4         15            7           19          14              22   
5         11            6           27          24              17   
6          1            0            2           0               1   
7          7            5           20          16              12   

   active count  total count  transition count duration per action  
0             2            3                 2             0:20:00  
1           179          201                32              0:0:17  
2            51           72                31              0:0:50  
3            45           55                14              0:1:05  
4            33           55                24              0:1:05  
5            51           68                16              0:0:52  
6             2            3                 2             0:20:00  
7            36           48                18              0:1:15

In [35]:
actual_from_all = []
for i in range(len(df_summary_all)):
    keep_start = -1
    
    for j in range(len(df_act_period)):
        floor_start = get_floor_start(df_act_period.loc[j, 'from'])
        
        if(df_act_period.loc[j, 'date']==df_summary_all.loc[i, 'date'] and 
           df_act_period.loc[j, 'ID']==df_summary_all.loc[i, 'ID']):

            if(floor_start>keep_start and 
               floor_start==calc_sec(df_summary_all.loc[i, 'from'])):
                
                actual_from_all.append(df_act_period.loc[j, 'from'])
                keep_start = calc_sec(df_act_period.loc[j, 'from'])

            elif(floor_start<=keep_start):
                break

In [133]:
actual_to_all = []
temp = 0
for i in range(len(df_summary_all)):
    keep_finish = calc_sec(df_act_period.loc[temp, 'to'])
    
    for j in range(temp, len(df_act_period)):
        
        ceil_finish = calc_sec(df_summary_all.loc[i, 'to'])
        
        
        if(df_act_period.loc[j, 'date']==df_summary_all.loc[i, 'date'] and 
           df_act_period.loc[j, 'ID']==df_summary_all.loc[i, 'ID']):

            if(calc_sec(df_act_period.loc[j, 'to'])>=keep_finish):
                
#                 actual_to_all.append(df_act_period.loc[j, 'to'])
                keep_finish = calc_sec(df_act_period.loc[j, 'to'])
            
            print(ceil_finish, calc_sec(df_act_period.loc[j, 'to']), j, temp)
        
            if(calc_sec(df_act_period.loc[j, 'to'])>ceil_finish):
                print('break')
                actual_to_all.append(df_act_period.loc[j-1, 'to'])
                temp = j
                break
                
            

50400.0 50362.811 0 0
50400.0 50398.15 1 0
50400.0 50400.0 2 0
50400.0 50436.696 3 0
break
54000.0 50436.696 3 3
54000.0 50438.143 4 3
54000.0 50443.602 5 3
54000.0 50443.923 6 3
54000.0 50444.566 7 3
54000.0 50444.726 8 3
54000.0 50446.975 9 3
54000.0 50447.617 10 3
54000.0 50447.778 11 3
54000.0 50454.524 12 3
54000.0 50461.59 13 3
54000.0 50462.073 14 3
54000.0 50463.839 15 3
54000.0 50467.372 16 3
54000.0 50469.46 17 3
54000.0 50473.476 18 3
54000.0 50474.119 19 3
54000.0 50481.83 20 3
54000.0 50482.792 21 3
54000.0 50482.953 22 3
54000.0 50483.435 23 3
54000.0 50486.166 24 3
54000.0 50487.612 25 3
54000.0 50493.073 26 3
54000.0 50493.876 27 3
54000.0 50524.105 28 3
54000.0 50526.968 29 3
54000.0 50528.895 30 3
54000.0 50533.717 31 3
54000.0 50535.162 32 3
54000.0 50564.559 33 3
54000.0 50565.683 34 3
54000.0 50568.735 35 3
54000.0 50570.342 36 3
54000.0 50571.305 37 3
54000.0 50579.981 38 3
54000.0 50614.362 39 3
54000.0 50622.074 40 3
54000.0 50622.395 41 3
54000.0 50627.216 42 3

46800.0 46801.524 457 3
break
50400.0 46801.524 457 457
50400.0 46801.846 458 457
50400.0 46802.649 459 457
50400.0 46839.751 460 457
50400.0 46880.709 461 457
50400.0 46881.029 462 457
50400.0 46881.67 463 457
50400.0 46881.834 464 457
50400.0 46886.008 465 457
50400.0 46886.168 466 457
50400.0 46886.648 467 457
50400.0 46895.48 468 457
50400.0 46898.535 469 457
50400.0 46901.584 470 457
50400.0 46944.632 471 457
50400.0 46988.639 472 457
50400.0 46991.366 473 457
50400.0 46992.491 474 457
50400.0 46992.812 475 457
50400.0 46994.901 476 457
50400.0 46996.187 477 457
50400.0 47001.005 478 457
50400.0 47032.966 479 457
50400.0 47033.449 480 457
50400.0 47033.61 481 457
50400.0 47035.858 482 457
50400.0 47041.156 483 457
50400.0 47043.886 484 457
50400.0 47050.31 485 457
50400.0 47053.842 486 457
50400.0 47084.356 487 457
50400.0 47084.517 488 457
50400.0 47088.05 489 457
50400.0 47088.209 490 457
50400.0 47089.174 491 457
50400.0 47089.334 492 457
50400.0 47098.327 493 457
50400.0 47130

In [131]:
actual_to_all

['14:0:0.0', '13:0:0.0']

In [74]:
df_summary_all['from actual'] = pd.Series(actual_from_all)
df_summary_all['to_actual'] = pd.Series(actual_to_all)

In [76]:
df_act_period

ID        date          from            to y_pred
0    2001  2019-01-16  13:58:47.956  13:59:22.811      2
1    2001  2019-01-16  13:59:22.972   13:59:58.15      0
2    2001  2019-01-16  13:59:58.311      14:0:0.0      2
3    2001  2019-01-16      14:0:0.0  14:00:36.696      2
4    2001  2019-01-16  14:00:36.857  14:00:38.143      3
5    2001  2019-01-16  14:00:38.302  14:00:43.602      2
6    2001  2019-01-16  14:00:43.763  14:00:43.923      3
7    2001  2019-01-16  14:00:44.115  14:00:44.566      2
8    2001  2019-01-16  14:00:44.726  14:00:44.726      3
9    2001  2019-01-16  14:00:44.887  14:00:46.975      2
10   2001  2019-01-16  14:00:47.146  14:00:47.617      3
11   2001  2019-01-16  14:00:47.778  14:00:47.778      2
12   2001  2019-01-16  14:00:47.939  14:00:54.524      0
13   2001  2019-01-16  14:00:54.684   14:01:01.59      3
14   2001  2019-01-16   14:01:01.75  14:01:02.073      2
15   2001  2019-01-16  14:01:02.234  14:01:03.839      0
16   2001  2019-01-16    14:01:04.0  14:01:07.372      1
17   2001  2019-01-16  14:01:07.539   14:01:09.46      3
18   2001  2019-01-16  14:01:09.622  14:01:13.476      0
19   2001  2019-01-16  14:01:13.637  14:01:14.119      3
20   2001  2019-01-16   14:01:14.28   14:01:21.83      0
21   2001  2019-01-16   14:01:21.99  14:01:22.792      1
22   2001  2019-01-16  14:01:22.953  14:01:22.953      0
23   2001  2019-01-16  14:01:23.114  14:01:23.435      2
24   2001  2019-01-16  14:01:23.595  14:01:26.166      0
25   2001  2019-01-16  14:01:26.326  14:01:27.612      3
26   2001  2019-01-16  14:01:27.772  14:01:33.073      2
27   2001  2019-01-16  14:01:33.234  14:01:33.876      3
28   2001  2019-01-16  14:01:34.037  14:02:04.105      1
29   2001  2019-01-16  14:02:04.237  14:02:06.968      0
..    ...         ...           ...           ...    ...
475  3005  2019-03-05  13:03:12.651  13:03:12.812      1
476  3005  2019-03-05  13:03:12.974  13:03:14.901      3
477  3005  2019-03-05  13:03:15.063  13:03:16.187      0
478  3005  2019-03-05  13:03:16.345  13:03:21.005      2
479  3005  2019-03-05  13:03:21.165  13:03:52.966      1
480  3005  2019-03-05  13:03:53.128  13:03:53.449      2
481  3005  2019-03-05   13:03:53.61   13:03:53.61      1
482  3005  2019-03-05  13:03:53.768  13:03:55.858      2
483  3005  2019-03-05  13:03:56.019  13:04:01.156      0
484  3005  2019-03-05  13:04:01.318  13:04:03.886      2
485  3005  2019-03-05  13:04:04.048   13:04:10.31      0
486  3005  2019-03-05  13:04:10.472  13:04:13.842      3
487  3005  2019-03-05  13:04:14.003  13:04:44.356      2
488  3005  2019-03-05  13:04:44.517  13:04:44.517      3
489  3005  2019-03-05  13:04:44.678   13:04:48.05      2
490  3005  2019-03-05  13:04:48.209  13:04:48.209      3
491  3005  2019-03-05  13:04:48.371  13:04:49.174      2
492  3005  2019-03-05  13:04:49.334  13:04:49.334      3
493  3005  2019-03-05  13:04:49.495  13:04:58.327      2
494  3005  2019-03-05  13:04:58.489  13:05:30.608      3
495  3005  2019-03-05  13:05:30.769  13:05:34.623      2
496  3005  2019-03-05  13:05:34.782  13:05:34.943      3
497  3005  2019-03-05  13:05:35.104  13:05:35.104      2
498  3005  2019-03-05  13:05:35.263  13:05:35.585      3
499  3005  2019-03-05  13:05:35.747  13:05:35.747      2
500  3005  2019-03-05  13:05:35.907  13:05:44.899      3
501  3005  2019-03-05  13:05:45.058  13:05:45.058      2
502  3005  2019-03-05   13:05:45.22  13:05:46.344      3
503  3005  2019-03-05  13:05:46.503  13:05:46.503      2
504  3005  2019-03-05  13:05:46.665  13:05:57.101      3

[505 rows x 5 columns]

In [75]:
df_summary_all

ID        date    from      to     sit   sleep   stand    walk   total  \
0  2001  2019-01-16  13:0:0  14:0:0  0:0:35  0:0:00  0:0:36  0:0:00  0:1:11   
1  2001  2019-01-16  14:0:0  15:0:0  0:1:07  0:1:10  0:4:03  0:3:04  0:9:24   
2  3001  2019-03-05  10:0:0  11:0:0  0:1:55  0:1:09  0:3:02  0:1:04  0:7:10   
3  3002  2019-03-05  12:0:0  13:0:0  0:1:06  0:1:10  0:2:48  0:0:30  0:5:34   
4  3003  2019-03-05  12:0:0  13:0:0  0:1:28  0:1:15  0:2:09  0:1:27  0:6:19   
5  3004  2019-03-05  12:0:0  13:0:0  0:1:53  0:1:18  0:2:44  0:0:56  0:6:51   
6  3005  2019-03-05  12:0:0  13:0:0  0:0:01  0:0:00  0:0:30  0:0:00  0:0:31   
7  3005  2019-03-05  13:0:0  14:0:0  0:1:44  0:1:17  0:1:47  0:0:59  0:5:47   

   sit count  sleep count  stand count  walk count  inactive count  \
0          1            0            2           0               1   
1         16            6           88          91              22   
2         12            9           32          19              21   
3          6            4           25          20              10   
4         15            7           19          14              22   
5         11            6           27          24              17   
6          1            0            2           0               1   
7          7            5           20          16              12   

   active count  total count  transition count duration per action  \
0             2            3                 2             0:20:00   
1           179          201                32              0:0:17   
2            51           72                31              0:0:50   
3            45           55                14              0:1:05   
4            33           55                24              0:1:05   
5            51           68                16              0:0:52   
6             2            3                 2             0:20:00   
7            36           48                18              0:1:15   

    from actual     to_actual  
0  13:58:47.956  13:59:22.811  
1      14:0:0.0      14:0:0.0  
2  10:03:24.717  10:04:06.297  
3  12:23:37.598  12:24:07.938  
4  12:33:59.899   12:34:29.77  
5  12:47:04.763  12:47:34.635  
6  12:59:27.632   12:59:57.52  
7      13:0:0.0      13:0:0.0

In [141]:
csv_path = datapath + 'summary/all_day_summary_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
df_summary_all.to_csv(csv_path)

In [143]:
csv_path = datapath + 'summary/activity_period_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
df_act_period.to_csv(csv_path)